In [1]:
import numpy as np
import pandas as pd

from math import *
from scipy.special import lpmn
# from scipy.special import legendre

# import seaborn as sns
# import matplotlib.pyplot as plt
# %matplotlib inline

## Input

In [2]:
# Jupiter dipole parameters

R0 = 7.1492e7 #m, Jupiter radius
b_eq = -4.2e5 #nT, field on planet equator

In [3]:
# Magnetodisk parameters

kmax = 20 # 20
pwr = 2 # 1/r^pwr magnetodisk, pwr=2 for [Alexeev and Belenkaya, 2005]
BDC = 2.5e-9 # 2.5e-9, magnetic field on edge RD1
RD2 = 18.4 # 18.4, disk inner edge
RD1 = 92.07 # 92.07, disk outer edge

In [4]:
sign_correction = 1

## Calculation

### Dipole field

In [5]:
def dip_field(r, R0, b_eq):
    # Planet dipole field calculation
    # INPUT:
    #   r [R0] - 1D-vector, length=3: (x,y,z)-coordinates of point in space
    #   R0 [m] - scalar: planet radius
    #   b_eq [nT] - scalar: magnetic on planet equator
    # OUTPUT:
    #   B [T] - 1D-vector, length=3: magnetic field
    
    r = R0 * np.array(r)
    M0 = -0.01 * b_eq * R0**3 # planet dipole magnetic moment (SI units, A*m^2)
    Mcoef = 1e-7 * M0;
    rr = sqrt(r[0] * r[0] + r[1] * r[1] + r[2] * r[2])
    Bx = 3 * Mcoef * r[0] * r[2] / rr**5
    By = 3 * Mcoef * r[1] * r[2] / rr**5
    Bz = Mcoef * (3 * r[2] * r[2] - rr**2) / rr**5
    B = np.array([Bx, By, Bz]);
    return B

In [6]:
dip_field([30, 0, 0], R0, b_eq)

array([ 0.00000000e+00,  0.00000000e+00, -1.55555556e-08])

### Magnetodisk field

In [7]:
def gen_a2k(kmax):
    # Coefficients a_2k

    a2kArr = np.zeros(kmax + 1);
    a2kArr[0] = 1
    if kmax > 0:
        for j in range(1, kmax + 1):
            a2kArr[j] = -a2kArr[j-1] * (2 * j - 1) / (2 * j)
    return a2kArr

In [8]:
a2kArr = gen_a2k(kmax)
print(a2kArr)

[ 1.         -0.5         0.375      -0.3125      0.2734375  -0.24609375
  0.22558594 -0.20947266  0.19638062 -0.18547058  0.17619705 -0.1681881
  0.16118026 -0.15498102  0.14944598 -0.14446445  0.13994993 -0.13583376
  0.1320606  -0.12858532  0.12537069]


In [9]:
def ser_field_uni_signum1(r, pwr, RD1, RD2, BDC, kmax, a2kArr, sign_correction):
    # Magnetodisk field: calculation through series for j~1/r^pwr
    # r - 1D-vector: (x,y,z)-coordinates
    # pwr - scalar: inverse power
    # RD1 - scalar: outer disk edge
    # RD2 - scalar: inner disk edge
    # kmax - scalar: sum number
    # a2kVec - 1D-vector: coefficients
    # sign_correction - scalar: correction for th extremely close to pi/2
    
    p = pwr #for brevity
    tau0 = RD2 / RD1
    BMDr = 0 # r-component of magnetodisk field in spherical coordinates
    BMDth = 0 # theta-component of magnetodisk field in spherical coordinates
    rr = sqrt(r[0] * r[0] + r[1] * r[1] + r[2] * r[2]) # |r|
    th = acos(r[2] / rr)
    tau = rr / RD1
    xi = cos(th)
    eta = sin(th)
    xiArr = np.zeros(2 * kmax + 2)
    xiArr[0] = 1
    xiArr[1] = xi
    for j in range(1, 2 * kmax):
        xiArr[j] = xi * xiArr[j]
    for k in range(kmax + 1):
        if p >= 3 and 2 * k == p - 3:
            specodd = True # odd special case: p=3, 5, 7,...
        else:
            specodd = False
        if p <= 0 and 2 * k == -p:
            speceven = True # even special case: p=0,-2,-4,...
        else:
            speceven = False
        a2k = a2kArr[k]
        allmlegendre = lpmn(1, 2*k+1, xi) # assosiated Legendre functions for all m
        P2kp1 = allmlegendre[0][0][-1] # equiv. alternative: P2kp1 = legendre(2*k+1)(xi)
        Q2kp1 = -allmlegendre[0][1][-1]
        # print(P2kp1)
        # print(Q2kp1)
        if rr <= RD2:
            if speceven:
                BMDr = BMDr - BDC * (2 * k + 1) * a2k * P2kp1 * log(tau0) / tau**p;
                BMDth = BMDth + BDC * a2k * Q2kp1 * log(tau0) / tau**p;
            else:
                BMDr = BMDr + BDC * (2 * k + 1) * a2k * P2kp1 * \
                    tau**(2 * k) * (1 / tau0**(2 * k + p) - 1)/(2 * k + p);
                BMDth = BMDth - BDC * (a2k * Q2kp1 / (2 * k + p)) * \
                    (1 / tau0**(2 * k + p) - 1) * tau**(2 * k)
        elif rr > RD2 and rr < RD1:
            if (not specodd) and (not speceven):
                psingBMDr = (BDC / tau**p) * \
                    (p - 1)**2 * (p - 2)**2 * a2k * P2kp1 * (4 * k + 3) / \
                    ((2 * k + 1) * (2 * k + 2)**2 * (2 * k + p) * (2 * k - p + 3)) # singular part-1
                BMDr = BMDr + psingBMDr + BDC * (2 * k + 1) * a2k * P2kp1 * \
                    (-tau**(2 * k) / (2 * k + p) - \
                    tau0**(2 * k - p + 3) / ((2 * k - p + 3) * tau**(2 * k + 3)))
                if p != 2:
                    psingBMDth = (BDC * (p - 2) / tau**p) * \
                        (p - 1) * (p - 2) * a2k * Q2kp1 * (4 * k + 3) / \
                        ((2 * k + 1) * (2 * k + 2)**2 * (2 * k + p) * (2 * k - p + 3)) # singular part-1
                    BMDth = BMDth + psingBMDth + BDC * (a2k * Q2kp1 / (2 * k + 2)) * \
                        ((2 * k + 2) * tau**(2 * k) / (2 * k + p) - \
                        (2 * k + 1) * tau0**(2 * k - p + 3) / \
                            ((2 * k - p + 3) * tau**(2 * k + 3)))
                else:
                    BMDth = BMDth + \
                        BDC * (a2k * Q2kp1 / (2 * k + 2)) * (tau**(2 * k) - \
                            tau0**(2 * k + 1) / tau**(2 * k + 3))
            elif specodd:
                msingBMDr = (BDC / tau**p) * \
                    a2k * P2kp1 * (4 * k + 3) / (2 * k + 2) * \
                    (1 + (p - 1) * (p - 2)* log(1 + abs(xi))) # minus singular part-0
                msingBMDth = (BDC * (p - 2) / tau**p) * \
                    a2k * Q2kp1 * (4 * k + 3) / ((2 * k + 1) * (2 * k + 2)**2) # minus singular part-0
                BMDr = BMDr - msingBMDr + BDC * (2 * k + 1) * a2k * P2kp1 * \
                    ((log(tau / tau0) + 1 / (2 * k + p)) / tau**p - \
                    tau**(2 * k) / (2 * k + p))
                BMDth = BMDth - msingBMDth + BDC * (a2k * Q2kp1 / (2 * k + 2)) * \
                    (((p - 2) * (log(tau / tau0) + 1 / (2 * k + p)) - 1) / tau**p + \
                    (2 * k + 2) * tau**(2 * k) / (2 * k + p))
            elif speceven:
                msingBMDr = (BDC / tau**p) * \
                    a2k * P2kp1 * (4 * k + 3) / (2 * k + 2) * \
                    (1 + (p - 1) * (p - 2) * log(1 + abs(xi))) # minus singular part-0
                msingBMDth = (BDC * (p - 2) / tau**p) * \
                    a2k * Q2kp1 * (4 * k + 3) / ((2 * k + 1) * (2 * k + 2)**2) # minus singular part-0
                BMDr = BMDr - msingBMDr + BDC * (2 * k + 1) * a2k * P2kp1 * \
                    ((1 / (2 * k - p + 3) - log(tau)) / tau**p - \
                    tau0**(2 * k - p + 3) / ((2 * k - p + 3) * tau**(2 * k + 3)))
                BMDth = BMDth - msingBMDth + BDC * (a2k * Q2kp1 /(2 * k + 2)) * \
                    (((p - 2) * (1 / (2 * k - p + 3) - log(tau)) + 1) / tau**p - \
                    (2 * k + 1) * tau0**(2 * k - p + 3) / \
                        ((2 * k - p + 3) * tau**(2 * k + 3)))
        elif rr >= RD1:
            if specodd:
                BMDr = BMDr - BDC * (2 * k + 1) * a2k * P2kp1 * log(tau0) / tau**p
                BMDth = BMDth - \
                    BDC * (a2k * Q2kp1 / (2 * k + 2)) * (2 * k + 1) * \
                    log(tau0) / tau**p
            else:
                BMDr = BMDr + BDC * (2 * k + 1) * a2k * P2kp1 * \
                    (1 - tau0**(2 * k - p + 3)) / \
                        ((2 * k - p + 3) * tau**(2 * k + 3));
                BMDth = BMDth + BDC * (a2k * Q2kp1 / (2 * k + 2)) * \
                    (2 * k + 1) * (1 - tau0**(2 * k - p + 3)) / \
                        ((2 * k - p + 3) * tau**(2 * k + 3))
    if rr > RD2 and rr < RD1:
        BMDr = BMDr + (BDC / tau**p) * np.sign(xi) * \
            (1 + (p - 1) * (p - 2) * log(1 + abs(xi))) # singular part-0
        if p != 2:
            BMDth = BMDth + (BDC * (p - 2) / tau**p) * \
                (1 - abs(xi)) / eta # singular part-0
    # Magnetic field vector in spherical coordinates
    spherB = np.array([BMDr, BMDth, 0])
    # Correction
    if sign_correction < 0 and th == math.pi / 2:
        spherB = spherB * np.array([-1, 1, 1])
    # Spherical --> Cartesian
    if r[1] >= 0:
        phi = acos(r[0] / sqrt(r[0] * r[0] + r[1] * r[1]))
    else:
        phi = 2 * pi - acos(r[0] / sqrt(r[0] * r[0] + r[1] * r[1]))
    Spher2decMat = np.array([[sin(th) * cos(phi), cos(th) * cos(phi), -sin(phi)],
                [sin(th) * sin(phi), cos(th) * sin(phi), cos(phi)],
                [cos(th), -sin(th), 0]])
    B = np.dot(Spher2decMat, spherB)
    return B

In [10]:
r = [30, 0, 1e-7]
B = dip_field(r, R0, b_eq) + ser_field_uni_signum1(r, pwr, RD1, RD2, BDC, kmax, a2kArr, sign_correction)
print(B)

[ 2.35469026e-08  0.00000000e+00 -8.28562966e-09]


## Samples

### rr > RD2 and rr < RD1

[Alexeev and Belenkaya, 2005]:

In [11]:
ser_field_uni_signum1([30, 0, 1e-7], pwr, RD1, RD2, BDC, kmax, a2kArr, 1)

array([2.35469024e-08, 0.00000000e+00, 7.26992589e-09])

In [12]:
print(ser_field_uni_signum1([30, 0, 1e-7], 0, RD1, RD2, BDC, kmax, a2kArr, 1))
print(ser_field_uni_signum1([30, 0, 1e-7], 1, RD1, RD2, BDC, kmax, a2kArr, 1))
print(ser_field_uni_signum1([30, 0, 1e-7], 1.9995, RD1, RD2, BDC, kmax, a2kArr, 1))
print(ser_field_uni_signum1([30, 0, 1e-7], 3, RD1, RD2, BDC, kmax, a2kArr, 1))

[2.50000000e-09 0.00000000e+00 3.79458711e-09]
[7.67249996e-09 0.00000000e+00 6.03261701e-09]
[2.35337039e-08 0.00000000e+00 7.27066632e-09]
[ 7.22654440e-08  0.00000000e+00 -4.74515974e-09]


### rr <= RD2

[Alexeev and Belenkaya, 2005]:

In [13]:
ser_field_uni_signum1([10, 0, 1e-7], pwr, RD1, RD2, BDC, kmax, a2kArr, 1)

array([1.06770674e-16, 0.00000000e+00, 3.43289288e-08])

In [14]:
print(ser_field_uni_signum1([10, 0, 1e-7], 0, RD1, RD2, BDC, kmax, a2kArr, 1))
print(ser_field_uni_signum1([10, 0, 1e-7], 1, RD1, RD2, BDC, kmax, a2kArr, 1))
print(ser_field_uni_signum1([10, 0, 1e-7], 1.9995, RD1, RD2, BDC, kmax, a2kArr, 1))
print(ser_field_uni_signum1([10, 0, 1e-7], 3, RD1, RD2, BDC, kmax, a2kArr, 1))

[7.46948285e-18 0.00000000e+00 4.33905955e-09]
[2.71648375e-17 0.00000000e+00 1.11201610e-08]
[1.06696124e-16 0.00000000e+00 3.43082950e-08]
[4.40875523e-16 0.00000000e+00 1.20995984e-07]


### rr >= RD1

[Alexeev and Belenkaya, 2005]:

In [15]:
ser_field_uni_signum1([100, 5, 1e-7], 2, RD1, RD2, BDC, kmax, a2kArr, 1)

array([ 1.01231425e-17,  5.06157127e-19, -1.59735670e-09])

In [16]:
print(ser_field_uni_signum1([100, 5, 1e-7], 0, RD1, RD2, BDC, kmax, a2kArr, 1))
print(ser_field_uni_signum1([100, 5, 1e-7], 1, RD1, RD2, BDC, kmax, a2kArr, 1))
print(ser_field_uni_signum1([100, 5, 1e-7], 1.9995, RD1, RD2, BDC, kmax, a2kArr, 1))
print(ser_field_uni_signum1([100, 5, 1e-7], 3, RD1, RD2, BDC, kmax, a2kArr, 1))

[ 7.12973779e-18  3.56486890e-19 -9.18041078e-10]
[ 8.24828829e-18  4.12414415e-19 -1.15363065e-09]
[ 1.01218805e-17  5.06094027e-19 -1.59704214e-09]
[ 1.39146412e-17  6.95732058e-19 -2.60133721e-09]
